# Observing Blocks

In [1]:
from keckODL.target import Target, TargetList
from keckODL.kcwi import KCWIblueDetectorConfig, KCWIredDetectorConfig
from keckODL.kcwi import KCWIConfig
from keckODL.offset import Stare, StarSkyStar
from keckODL.block import ObservingBlock, ObservingBlockList

from astropy import units as u

# KCWI + KCRM Example

In [2]:
ngc1333 = Target('NGC1333', rotmode='PA', PA=22.5,
                 objecttype='science',
                 acquisition='blind')

In [3]:
# Let's set up detector configurations for the red and blue arm
# For the blue side we'll take a single 1800s exposure
# For the red side, we'll do two 900s exposures
kcwib_1800s = KCWIblueDetectorConfig(exptime=1800, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
kcwir_1800s = KCWIredDetectorConfig(exptime=900, nexp=2, binning='2x2', readoutmode=0, ampmode=9)
kcwib_1800s.write('detector_KCWIb_1800s.txt')
kcwir_1800s.write('detector_KCWIr_1800s.txt')

In [4]:
# How long will the blue side exposure take with overheads?
kcwib_1800s.estimate_clock_time()

1853

In [5]:
# How long will the red side exposure take with overheads?
kcwir_1800s.estimate_clock_time()

1800

In [6]:
# Let's ask the red side to adjust the exposure time to 
# match the time it takes for the blue side to finish
kcwir_1800s.match_time(kcwib_1800s.estimate_clock_time())
kcwir_1800s.exptime

926.5

In [7]:
# Now the red side will finish at the same time the blue side does
kcwir_1800s.estimate_clock_time()

1853.0

In [8]:
med_slicer_b4800_r6500 = KCWIConfig(slicer='medium', bluegrating='BH3', bluecwave=4800,
                                    redgrating='RH3', redcwave=6563,)
med_slicer_b4800_r6500.write('KCWIconfig_medslicer_red_blue.txt')
med_slicer_b4800_r6500

medium BH3 4800 A

In [9]:
skyoffset = StarSkyStar(dx=60*u.arcsec, dy=30*u.arcsec, repeat=3)
skyoffset.write('offset_customStarSkyStar.txt')

In [10]:
OB1 = ObservingBlock(target=ngc1333, pattern=skyoffset,
                            detconfig=(kcwib_1800s, kcwir_1800s),
                            instconfig=med_slicer_b4800_r6500,
                            )

In [11]:
OBlist = ObservingBlockList([OB1])
OBlist

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
NGC1333        |StarSkyStar (60 30) x3|(KCWIblue 1800s x1, KCWIred 926s x2)|medium BH3 4800 A                            

In [12]:
OBlist.estimate_time()

Shutter Open Time: 16677.0 s (4.6 hrs)
Wall Clock Time: 16677 s (4.6 hrs)


In [13]:
cals_I_need = OBlist.cals()
cals_I_need

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
None           |Stare x1              |KCWIblue 6s x1                      |medium BH3 4800 A arclamp=CONT calobj=MedBarsA
None           |Stare x1              |KCWIblue 30s x1                     |medium BH3 4800 A arclamp=FEAR calobj=FlatA  
None           |Stare x1              |KCWIblue 45s x1                     |medium BH3 4800 A arclamp=THAR calobj=FlatA  
None           |Stare x6              |KCWIblue 6s x1                      |medium BH3 4800 A arclamp=CONT calobj=FlatA  
None           |Stare x7              |KCWIblue 0s (Dark) x1               |medium BH3 4800 A                            
DomeFlats      |Stare x3              |KCWIblue 100s x1                    |medium BH3 4800 A domeflatlamp=True          

In [14]:
cals_I_need.estimate_time()

Shutter Open Time: 417 s (0.1 hrs)
Wall Clock Time: 3647 s (1.0 hrs)


## MOSFIRE Example

In [15]:
from keckODL.target import Target, TargetList
from keckODL.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, long2pos, ABBA
from keckODL.offset import Stare
from keckODL.block import ObservingBlock, ObservingBlockList

In [16]:
v1647Ori = Target('V1647 Ori', rotmode='PA', PA=22.5,
                  objecttype='science',
                  acquisition='blind')

In [17]:
# For MOSFIRE we leave nexp=1 in the detector configs because we only
# want one exposure at each point in the dither pattern.
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
print(mosfire_180s)
print(mosfire_120s)

MOSFIRE 180s (MCDS16, 1 coadds) x1
MOSFIRE 120s (MCDS16, 1 coadds) x1


In [18]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

my_MOS_mask Y-spectroscopy
my_MOS_mask J-spectroscopy
my_MOS_mask H-spectroscopy
my_MOS_mask K-spectroscopy


In [19]:
abba5 = ABBA(offset=1.25*u.arcsec, repeat=5)
print(abba5)
abba8 = ABBA(offset=1.25*u.arcsec, repeat=8)
print(abba8)

ABBA (1.25 arcsec) x5
ABBA (1.25 arcsec) x8


In [20]:
abba5

Frame: MOSFIRE Slit (INSTXOFF INSTYOFF)
Repeats: 5
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
  +0.0|  +1.2|    +0.0|       A|  True
  +0.0|  -1.2|    +0.0|       B|  True
  +0.0|  -1.2|    +0.0|       B|  True
  +0.0|  +1.2|    +0.0|       A|  True

In [21]:
for i,offset in enumerate(abba5):
    print(f'Executing offset: {i}')
    offset.execute()

Executing offset: 0
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(1.25, rel2base=t)
Executing offset: 1
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(-1.25, rel2base=t)
Executing offset: 2
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(-1.25, rel2base=t)
Executing offset: 3
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(1.25, rel2base=t)


In [22]:
myOBs = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba5, detconfig=mosfire_180s, instconfig=mosfire_Y),
                            ObservingBlock(target=v1647Ori, pattern=abba8, detconfig=mosfire_120s, instconfig=mosfire_J),
                            ObservingBlock(target=v1647Ori, pattern=abba8, detconfig=mosfire_120s, instconfig=mosfire_H),
                            ObservingBlock(target=v1647Ori, pattern=abba5, detconfig=mosfire_180s, instconfig=mosfire_K),
                           ])
print(myOBs)

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
V1647 Ori      |ABBA (1.25 arcsec) x5 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |my_MOS_mask Y-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x8 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |my_MOS_mask J-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x8 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |my_MOS_mask H-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x5 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |my_MOS_mask K-spectroscopy                   


In [23]:
myOBs.estimate_time()

Shutter Open Time: 14880 s (4.1 hrs)
Wall Clock Time: 14880 s (4.1 hrs)


In [24]:
cals_I_need = mosfire_Y.cals()
cals_I_need.extend( mosfire_J.cals() )
cals_I_need.extend( mosfire_H.cals() )
cals_I_need.extend( mosfire_K.cals() )
print(cals_I_need)

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask Y-spectroscopy domeflatlamp=True 
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask J-spectroscopy domeflatlamp=True 
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask H-spectroscopy domeflatlamp=True 
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask K-spectroscopy domeflatlamp=True 
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask K-spectroscopy domeflatlamp=False
None           |Stare x2              |MOSFIRE 1s (CDS, 1 coadds) x1       |my_MOS_mask K-spectroscopy arclamp=Ne        
None           |Stare x2

In [25]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)


## MOSFIRE Example 2

In [26]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(singleobj_Y)
print(singleobj_J)
print(singleobj_H)
print(singleobj_K)

longslit_3x0.7 Y-spectroscopy
longslit_3x0.7 J-spectroscopy
long2pos H-spectroscopy
long2pos K-spectroscopy


In [27]:
abba2 = ABBA(offset=1.25*u.arcsec, repeat=2)
print(abba2)
long2pos_pattern = long2pos(repeat=2)
print(long2pos_pattern)

ABBA (1.25 arcsec) x2
long2pos x2


In [28]:
long2pos_pattern

Frame: MOSFIRE Detector (INSTXOFF INSTYOFF)
Repeats: 2
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
 +45.0| -23.0|    +0.0|       A|  True
 +45.0|  -9.0|    +0.0|       B|  True
 -45.0|  +9.0|    +0.0|       A|  True
 -45.0| +23.0|    +0.0|       B|  True

In [29]:
long2pos_pattern[0].execute()

'INSTXOFF'.write(45.0, rel2base=t)
'INSTYOFF'.write(-23.0, rel2base=t)


In [30]:
myOBs2 = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba2, detconfig=mosfire_180s, instconfig=singleobj_Y),
                             ObservingBlock(target=v1647Ori, pattern=abba2, detconfig=mosfire_120s, instconfig=singleobj_J),
                             ObservingBlock(target=v1647Ori, pattern=long2pos_pattern, detconfig=mosfire_120s, instconfig=singleobj_H),
                             ObservingBlock(target=v1647Ori, pattern=long2pos_pattern, detconfig=mosfire_180s, instconfig=singleobj_K),
                            ])
print(myOBs2)

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
V1647 Ori      |ABBA (1.25 arcsec) x2 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |longslit_3x0.7 Y-spectroscopy                
V1647 Ori      |ABBA (1.25 arcsec) x2 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |longslit_3x0.7 J-spectroscopy                
V1647 Ori      |long2pos x2           |MOSFIRE 120s (MCDS16, 1 coadds) x1  |long2pos H-spectroscopy                      
V1647 Ori      |long2pos x2           |MOSFIRE 180s (MCDS16, 1 coadds) x1  |long2pos K-spectroscopy                      


In [31]:
myOBs2.estimate_time()

Shutter Open Time: 4800 s (1.3 hrs)
Wall Clock Time: 4800 s (1.3 hrs)


In [32]:
cals_I_need = singleobj_Y.cals()
cals_I_need.extend(singleobj_J.cals())
cals_I_need.extend(singleobj_H.cals())
cals_I_need.extend(singleobj_K.cals())
print(cals_I_need)

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |longslit_3x0.7 Y-spectroscopy domeflatlamp=True
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |longslit_3x0.7 J-spectroscopy domeflatlamp=True
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |long2pos H-spectroscopy domeflatlamp=True    
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |long2pos K-spectroscopy domeflatlamp=True    
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |long2pos K-spectroscopy domeflatlamp=False   
None           |Stare x2              |MOSFIRE 1s (CDS, 1 coadds) x1       |long2pos K-spectroscopy arclamp=Ne           
None           |Star

In [33]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)


# MOSFIRE Night Example

In [34]:
myOBs3 = ObservingBlockList([ObservingBlock(target=ngc1333, pattern=abba2, detconfig=mosfire_180s, instconfig=singleobj_Y),
                             ObservingBlock(target=ngc1333, pattern=abba2, detconfig=mosfire_120s, instconfig=singleobj_J),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, detconfig=mosfire_120s, instconfig=singleobj_H),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, detconfig=mosfire_180s, instconfig=singleobj_K),
                            ])
print(myOBs3)

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
NGC1333        |ABBA (1.25 arcsec) x2 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |longslit_3x0.7 Y-spectroscopy                
NGC1333        |ABBA (1.25 arcsec) x2 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |longslit_3x0.7 J-spectroscopy                
NGC1333        |long2pos x2           |MOSFIRE 120s (MCDS16, 1 coadds) x1  |long2pos H-spectroscopy                      
NGC1333        |long2pos x2           |MOSFIRE 180s (MCDS16, 1 coadds) x1  |long2pos K-spectroscopy                      


In [35]:
myOBs.extend(myOBs2)
myOBs.extend(myOBs3)

In [36]:
myOBs

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
V1647 Ori      |ABBA (1.25 arcsec) x5 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |my_MOS_mask Y-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x8 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |my_MOS_mask J-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x8 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |my_MOS_mask H-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x5 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |my_MOS_mask K-spectroscopy                   
V1647 Ori      |ABBA (1.25 arcsec) x2 |MOSFIRE 180s (MCDS16, 1 coadds) x1  |longslit_3x0.7 Y-spectroscopy                
V1647 Ori      |ABBA (1.25 arcsec) x2 |MOSFIRE 120s (MCDS16, 1 coadds) x1  |longslit_3x0.7 J-spectroscopy                
V1647 Ori      |long2pos

In [37]:
myOBs.cals()

Target         |Pattern               |DetectorConfig                      |InstrumentConfig                             
---------------|----------------------|------------------------------------|---------------------------------------------
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask K-spectroscopy domeflatlamp=True 
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |my_MOS_mask K-spectroscopy domeflatlamp=False
None           |Stare x2              |MOSFIRE 1s (CDS, 1 coadds) x1       |my_MOS_mask K-spectroscopy arclamp=Ne        
None           |Stare x2              |MOSFIRE 1s (CDS, 1 coadds) x1       |my_MOS_mask K-spectroscopy arclamp=Ar        
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |long2pos H-spectroscopy domeflatlamp=True    
DomeFlats      |Stare x7              |MOSFIRE 11s (CDS, 1 coadds) x1      |longslit_3x0.7 J-spectroscopy domeflatlamp=True
DomeFlats      |Stare 